In [77]:
# 가상주소 -> 물리주소 변환
# 가상주소 범위 가져오기
# 물리주소 범위 지정하기 (가상주소 범위보다 당연히 훨씬 넓겠지ㅇㅅㅇ) => 일단 va 범위의 4배
# 가상주소 범위 huge page 단위로 잘라 물리주소에 mapping
# page size : 4KB -> 16KB -> 1MB

In [78]:
import pandas as pd
import numpy as np
import random
random.seed(1234)

In [79]:
va_df = pd.read_csv("cc_64.vaddr", names=['va'])

In [80]:
va_trace = va_df['va']

In [81]:
va_range = {"min":va_df.min()['va'], "max":va_df.max()['va']}

In [82]:
VIRTUAL_ADDRESS_SPACE_SIZE = va_range['max'] - va_range['min'] + 1
PHYSICAL_ADDRESS_SPACE_SIZE = VIRTUAL_ADDRESS_SPACE_SIZE * 4

In [83]:
print(VIRTUAL_ADDRESS_SPACE_SIZE, PHYSICAL_ADDRESS_SPACE_SIZE)

279864561094593 1119458244378372


In [84]:
PAGE_SIZE = 1 << 20

In [85]:
V_PAGE_NUM = VIRTUAL_ADDRESS_SPACE_SIZE // PAGE_SIZE
P_PAGE_NUM = PHYSICAL_ADDRESS_SPACE_SIZE // PAGE_SIZE
print(V_PAGE_NUM, P_PAGE_NUM)

266899643 1067598575


In [86]:
def make_pagetable(unique_vn):
    pn_candidate = random.sample(range(P_PAGE_NUM), len(unique_vn))
    page_table=pd.DataFrame({'vn':list(unique_vn), 'pn':pn_candidate})
    return page_table

In [87]:
unique_va = list(set(va_trace))
ununique_vn = list(x//PAGE_SIZE for x in unique_va)
unique_vn = set(ununique_vn)
# pn_candidate = random.sample(range(P_PAGE_NUM), len(unique_vn))

In [88]:
ununique_vn

[169524608,
 212114852,
 148822662,
 15343716,
 206097040,
 207508144,
 113701350,
 243283430,
 132887496,
 107321512,
 260344783,
 113701349,
 243283429,
 46320564,
 179498228,
 223131971,
 192595881,
 222030249,
 207508145,
 180804024,
 216086607,
 260344783,
 255580119,
 200793438,
 162400102,
 179498229,
 68347708,
 223131972,
 225421651,
 73700251,
 15343716,
 169524608,
 148822662,
 212114852,
 207508144,
 113701350,
 243283430,
 132887496,
 107321512,
 119365438,
 85021489,
 180804025,
 260344783,
 73710720,
 179498228,
 108731444,
 153029934,
 225421651,
 177735970,
 85021488,
 186675494,
 27724454,
 1513932,
 183793676,
 225421652,
 15343716,
 169524608,
 119365438,
 212114852,
 206097040,
 207508144,
 113701350,
 243283430,
 148822662,
 107321512,
 169524609,
 263176106,
 13161089,
 69011945,
 1513933,
 74089620,
 212114851,
 15343715,
 198894763,
 168038176,
 243283428,
 25936886,
 89942334,
 148822661,
 191235433,
 15343716,
 169524608,
 206097040,
 119365438,
 212114852,
 

In [89]:
unique_vn

{735258,
 1513932,
 1513933,
 5350972,
 10621729,
 12695024,
 13161089,
 15190259,
 15343714,
 15343715,
 15343716,
 18753565,
 18753566,
 18852374,
 20163868,
 20163869,
 25936886,
 25936887,
 27282263,
 27676613,
 27724454,
 30987738,
 30987739,
 30987740,
 31141574,
 34904684,
 38626648,
 39142146,
 39142147,
 39303814,
 39303815,
 40309699,
 41480807,
 42826995,
 42894204,
 42894205,
 46320564,
 46320565,
 48263052,
 50978012,
 52400747,
 53203275,
 57464206,
 57464207,
 57464208,
 60834930,
 68347708,
 68347709,
 69011945,
 73700250,
 73700251,
 73710720,
 74089620,
 74089621,
 74717796,
 74717797,
 74717798,
 77156522,
 80919367,
 85021488,
 85021489,
 85021490,
 88695458,
 89942334,
 91598853,
 92445294,
 100516926,
 104194041,
 107321511,
 107321512,
 107321513,
 108530502,
 108731443,
 108731444,
 109606747,
 111008059,
 112172397,
 112172398,
 112172399,
 113587971,
 113701348,
 113701349,
 113701350,
 119344472,
 119365437,
 119365438,
 121784133,
 127820524,
 127820525,
 13

In [90]:
pg_tb = make_pagetable(unique_vn)

In [91]:
pg_tb

,vn,pn
0,91598853,1037721582
1,184504327,835549143
2,183793676,473233031
3,183793677,125467289
4,18852374,8043909
...,...,...
198,25936886,40796390
199,25936887,235569931
200,104194041,544080912
201,142138874,890799985


In [92]:
1<<20

1048576

In [93]:
def v2p(page_table, va_trace):
    trace_list = []
    trace_df = pd.DataFrame()
    for i, va in enumerate(va_trace):
        vpn = va // PAGE_SIZE
        idx = va % PAGE_SIZE
        ppn = page_table[page_table['vn']==vpn].iloc[0]['pn']
        pa = ppn * PAGE_SIZE + idx
        trace_list.append({'va':va, 'pa':pa})
        # if (i % 1000000 == 0):
    trace_df = pd.DataFrame(trace_list)
    trace_df.to_csv("cc_MB.csv", index=False)
    return trace_df

In [94]:
v2p_result = v2p(pg_tb, va_trace)

In [95]:
v2p_result

,va,pa
0,44977833418496,584581451915008
1,44977833422656,584581451919168
2,44977833429056,584581451925568
3,44977833428800,584581451925312
4,44977833428928,584581451925440
...,...,...
9999995,16089051540736,888079131061504
9999996,16089051542144,888079131062912
9999997,16089051543808,888079131064576
9999998,137144013441664,634001173006976


In [96]:
## v2p
v2p_result['pa'].to_csv("cc_1MB.paddr", index=False)